### References <br>
https://medium.com/towards-data-science/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021 <br>
https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-2-bf2403804ada <br>


# Data

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=14TR8qzYX9QQPxpoRr779axbmpp0REGeP'
output = 'fhir-chunk-1.csv'
gdown.download(url, output, quiet=False)

In [ ]:
# import torch dataset class
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from glob import glob
import os
from datetime import datetime
import random

In [ ]:
#Write a DataSet Class that returns a FHIR resource as a PyTorch DataSet

class FHIRData(Dataset):
    def __init__(self, data_frame_path):
        if os.path.isfile(data_frame_path):
            self.data_frame = pd.read_csv(data_frame_path)
        elif os.path.isdir(data_frame_path):
            self.data_frame = pd.concat([pd.read_csv(os.path.join(data_frame_path, file)) for file in os.listdir(data_frame_path)])
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        row = self.data_frame.iloc[idx]
        return row['seed'], row['FHIR']

In [ ]:
# data = FHIRData('data\\time_seed_fhir.csv')

In [ ]:
from transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

df = pd.read_csv('data/time_seed_fhir.csv')
max_seed_length = len(tokenizer(df['seed'].iloc[np.argmax([len(x) for x in df['seed']])])['input_ids'])
max_fhir_length = len(tokenizer(df['FHIR'].iloc[np.argmax([len(x) for x in df['FHIR']])])['input_ids'])

tokenized_seed = tokenizer(list(df['seed'].iloc[:100]), max_length=max_seed_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']
tokenized_fhir = tokenizer(list(df['FHIR'].iloc[:100]), max_length=max_fhir_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']

print(tokenized_seed['input_ids'].shape, tokenized_fhir['input_ids'].shape)

In [ ]:
# tokenizer.add_tokens(list_of_new_tokens)

# # resize the embeddings
#  model.resize_token_embeddings(len(tokenizer))


# https://discuss.huggingface.co/t/t5-for-conditional-generation-getting-started/1284/8

# https://discuss.huggingface.co/t/t5-finetuning-tips/684



# Architectures

## Model 1: Custom Transformer

#### v1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

        self.device = device

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(self.device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
dropout = 0.1
max_seq_length = max(max_fhir_length, max_seed_length)

config = {
    'src_vocab_size': src_vocab_size,
    'tgt_vocab_size': tgt_vocab_size,
    'd_model': d_model,
    'num_heads': num_heads,
    'num_layers': num_layers,
    'd_ff': d_ff,
    'dropout': dropout,
    'max_seq_length': max_seq_length,
    'max_fhir_length': max_fhir_length,
    'max_seed_length': max_seed_length
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device).to(device)

# Generate random sample data
src_data = tokenized_seed['input_ids'].to(device)
tgt_data = tokenized_fhir['input_ids'].to(device)

In [ ]:
src_data.shape, tgt_data.shape

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [ ]:
from joblib import dump
torch.save(transformer.state_dict(), 'model.pt')
dump(config, 'config.joblib')

##### Evaluation

In [ ]:
from joblib import load
from transformers import XLNetTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import pandas as pd
from datetime import datetime
import random


config = load('config.joblib')

src_vocab_size = config['src_vocab_size']
tgt_vocab_size = config['tgt_vocab_size']
d_model = config['d_model']
num_heads = config['num_heads']
num_layers = config['num_layers']
d_ff = config['d_ff']
max_seq_length = config['max_seq_length']
dropout = config['dropout']
max_fhir_length = config['max_fhir_length']
max_seed_length = config['max_seed_length']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device).to(device)
model.load_state_dict(torch.load('model.pt'))

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

n_seeds = 1024
seeds = [datetime.now().strftime("%Y:%m:%d %H:%M:%S:%f") + " " + str(random.randint(0, 100_000_000_000_00)) for x in range(n_seeds)]


In [ ]:
tokenized_seeds = tokenizer(seeds, max_length=max_seed_length, padding='max_length', truncation=True, return_tensors="pt")

In [ ]:
tokenized_seeds

#### v2

In [ ]:
# https://pytorch.org/tutorials/beginner/translation_transformer.html

In [ ]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
import pandas as pd
import numpy as np

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:

from transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
vocab = tokenizer.get_vocab()

In [ ]:
PAD_IDX = tokenizer.all_special_tokens.index(tokenizer.pad_token)

In [ ]:
train_batch_size = 8
test_batch_size = 8
max_seed_length = 24
max_fhir_length = 2048
DEVICE = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv('data/time_seed_fhir_train.csv')

tokenized_seed = tokenizer(list(df['seed'].iloc[:100]), max_length=max_seed_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']
tokenized_fhir = tokenizer(list(df['FHIR'].iloc[:100]), max_length=max_fhir_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']

train_data = TensorDataset(tokenized_seed, tokenized_fhir)
train_data_loader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True)

print(tokenized_seed.shape, tokenized_fhir.shape)

In [ ]:
df = pd.read_csv('data/time_seed_fhir_test.csv')

tokenized_seed = tokenizer(list(df['seed'].iloc[:100]), max_length=max_seed_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']
tokenized_fhir = tokenizer(list(df['FHIR'].iloc[:100]), max_length=max_fhir_length, padding='max_length', truncation=True, return_tensors="pt")['input_ids']

test_data = TensorDataset(tokenized_seed, tokenized_fhir)
test_data_loader = DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

print(tokenized_seed.shape, tokenized_fhir.shape)

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab)
TGT_VOCAB_SIZE = len(vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 1
NUM_DECODER_LAYERS = 1

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
X, y = train_data_loader.__iter__().__next__()
X = X.to(device).transpose(0, 1)
y = y.to(device).transpose(0, 1)

In [ ]:
X.shape, y.shape

In [ ]:
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(X, y[:-1, :])

In [ ]:
out = transformer(X, y[:-1, :], src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

In [ ]:
out = model(X)

In [ ]:
y.shape

In [ ]:
out.shape

In [ ]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))

In [ ]:
def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


In [ ]:
# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

## Model 2: Fine Tuning Pre-trained BERT

In [ ]:
from datasets import load_dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import BartModel, BartTokenizer

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

In [ ]:
base_dir = ""
# If colab
# base_dir = '/content/'
data_path = base_dir + 'data/time_seed_fhir.csv'

In [ ]:
fhir_dataset = load_dataset('csv', data_files=data_path, split='train')
fhir_dataset = fhir_dataset.train_test_split(test_size=0.2)
print(f"Training Samples: {fhir_dataset['train'].num_rows}, Test Samples: {fhir_dataset['test'].num_rows}")

In [ ]:
prefix = "Generate a FHIR data row using the seed: "
# FHIR,seed
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["seed"]]
    model_inputs = tokenizer(inputs, truncation=True, padding="longest")
    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["FHIR"], truncation=True, padding="longest")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = fhir_dataset.map(preprocess_function, batched=True).remove_columns(['seed', 'FHIR'])

In [ ]:
model = BartModel.from_pretrained('facebook/bart-large', device_map="auto")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

training_args = Seq2SeqTrainingArguments(
    output_dir                  = "./results",
    evaluation_strategy         = "epoch",
    learning_rate               = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size  = 32,
    weight_decay                = 0.01,
    save_total_limit            = 3,
    num_train_epochs            = 1,
    fp16                        = True,
)

trainer = Seq2SeqTrainer(
    model         = model,
    args          = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset  = tokenized_dataset['test'],
    tokenizer     = tokenizer,
    data_collator = data_collator
)

In [ ]:
trainer.train()

## Model 3: Fine Tuning Pre-trained T5

In [ ]:
# https://huggingface.co/docs/transformers/main_classes/optimizer_schedules

# https://discuss.huggingface.co/t/t5-finetuning-tips/684/3

In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

In [ ]:
base_dir = ""
# If colab
# base_dir = '/content/'
data_path = base_dir + 'data/time_seed_fhir.csv'

In [ ]:
fhir_dataset = load_dataset('csv', data_files=data_path, split='train')
fhir_dataset = fhir_dataset.train_test_split(test_size=0.2)
print(f"Training Samples: {fhir_dataset['train'].num_rows}, Test Samples: {fhir_dataset['test'].num_rows}")

In [ ]:
prefix = "Generate a FHIR data row using the seed: "
# FHIR,seed
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["seed"]]
    model_inputs = tokenizer(inputs, truncation=True, padding="longest")
    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["FHIR"], truncation=True, padding="longest")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = fhir_dataset.map(preprocess_function, batched=True).remove_columns(['seed', 'FHIR'])

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")

# input_text = "[resourceType] CarePlan [id] 76d3b4b2-398f-4373-4957-599959c26e87 [meta][profile][0] http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan [text][status] generated [text][div] <div xmlns=""http://www.w3.org/1999/xhtml"">Care Plan for Infectious disease care plan (record artifact).<br/>Activities: <ul><li>Infectious disease care plan (record artifact)</li><li>Infectious disease care plan (record artifact)</li></ul><br/>Care plan is meant to treat COVID-19.</div> [status] completed [intent] order [category][0][coding][0][system] http://hl7.org/fhir/us/core/CodeSystem/careplan-category [category][0][coding][0][code] assess-plan [category][1][coding][0][system] http://snomed.info/sct [category][1][coding][0][code] 736376001 [category][1][coding][0][display] Infectious disease care plan (record artifact) [category][1][text] Infectious disease care plan (record artifact) [subject][reference] Patient/4a62b5fe-0dbd-fef9-e9a9-c21cecc5df61 [encounter][reference] Encounter/3bffc09e-9247-6538-e8db-6c119e3f1e2c [period][start] 2020-03-09T16:39:52-04:00 [period][end] 2020-03-27T13:39:52-04:00 [careTeam][0][reference] CareTeam/51954d07-f4a7-0f1a-b7a3-9c0f1f592038 [addresses][0][reference] Condition/3e78ad09-6fec-ede1-b27a-47e79bee28f1 [activity][0][detail][code][coding][0][system] http://snomed.info/sct [activity][0][detail][code][coding][0][code] 409524006 [activity][0][detail][code][coding][0][display] Airborne precautions (procedure) [activity][0][detail][code][text] Airborne precautions (procedure) [activity][0][detail][status] completed [activity][0][detail][location][display] BERKSHIRE MEDICAL CENTER INC - 1 [activity][1][detail][code][coding][0][system] http://snomed.info/sct [activity][1][detail][code][coding][0][code] 361235007 [activity][1][detail][code][coding][0][display] Isolation of infected patient (procedure) [activity][1][detail][code][text] Isolation of infected patient (procedure) [activity][1][detail][status] completed [activity][1][detail][location][display] BERKSHIRE MEDICAL CENTER INC - 1"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

training_args = Seq2SeqTrainingArguments(
    output_dir                  = "./results",
    evaluation_strategy         = "epoch",
    learning_rate               = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size  = 32,
    weight_decay                = 0.01,
    save_total_limit            = 3,
    num_train_epochs            = 1,
    fp16                        = True,
)

trainer = Seq2SeqTrainer(
    model         = model,
    args          = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset  = tokenized_dataset['test'],
    tokenizer     = tokenizer,
    data_collator = data_collator
)

In [ ]:
trainer.train()

## Model 4: Messing with a locally deployed LLM - 1 (Flan)

In [ ]:
# https://huggingface.co/declare-lab/flan-alpaca-gpt4-xl

## Model 4: Messing with a locally deployed LLM - 2 (llama2)